# DPO Simplified

Direct Preference Optimization (DPO) is very promising method of using preference data to train language models.

The goal of this article is to detail the core logic of DPO, namely that it represents straightforward maximum likelihood estimation of a penalized reward, under a simple pairwise comparison model.

The DPO paper heavily refers to a pairwise comparison model called the Bradley-Terry Model, so here I motivate and present the BT model before moving into the preference optimization setting.

I show how RLHF emerges from maximum likelihood estimation of an unconstrained reward under the B-T model.

Then, we show how DPO emerges from the same process on a constrained reward, and the nice theoretical analysis that results.

Finally, I'll briefly discuss the missing pre-training gradient mixin and some potential downstream consequences, as well as the limited evidence to-date.

This article is current work-in-progress.

## 1. Probabilities over Pairwise Comparison

Let's consider the following scenario - we have win-loss data from 100 tennis matches among 4 players: A, B, C and D.

Data in grid format: row A, column B shows wins by player A against player B, and so on.
||A|B|C|D|
|---|---|---|---|---|
|A|-|2|5|3|
|B|1|-|12|23|
|C|17|3|-|4|
|D|13|10|7|-|


Overall win-loss
|Player|W-L|
|---|---|
|A|10-31|
|B|36-15|
|C|24-24|
|D|30-30|

We want to evaluate the probability that A beats B in their next match - how should we approach this?

Let's define the random variable $W_{XY}$ which represents the probability that player X beats player Y, where $X,Y \in \{A,B,C,D\}$

## 2. Problems with head-to-head and overall win-rate

What we see from the grid data above is that A leads the head to head against B 2-1.

The winrate of A against B is an unbiased estimator for $W_{AB}$. However, we have just 3 matches for this, meaning it has high variance (under an assumption of somewhat bounded variance) compared to say $W_{BC}$ where we have 22 matches of data.

Moreover, B has a 12-3 record against C, and a 23-10 record against D, whereas A has a 5-17 record against C and a 3-13 record against D. This data paints the picture that B is a significantly better player than A, contradicting the limited head to head data. Overall winrates also suggest that B is the strongest player, and A is the weakest player.

However, overall winrates have a different issue - C and D have equivalent winrates, but D has played many matches against the strong B player and leads the head to head against C 7-4.

Direct head-to-head and WL both have flaws - we need a different model to predict winrates.

## 3. The Bradley-Terry Model

The BT model is a very simple model for pairwise comparison.

It assigns a score $p_X$ to each player, then models $P(X>Y)=\dfrac{p_X}{p_X+p_Y}$

In this case, we need $p_X>0$ $ \forall X$. 

If the scores can be negative, there is a relatively simple adaptation:

$P(X>Y)=\dfrac{e^{b_X}}{e^{b_X}+e^{b_Y}}$ i.e. $p_X=e^{b_X}$

The idea of BT is that we can learn the scores $p_X$ through straightforward maximum likelihood estimation on our given data.

The MLE update rule is given by $$p_X^{new}=\dfrac{\sum_Y w_{XY}p_Y/(p_X+p_Y)}{\sum_Y w_{YX}/(p_X+p_Y)}$$ where $w_{XY}$ represents the number of times $X$ beat $Y$ (see [here](https://jmlr.org/papers/volume24/22-1086/22-1086.pdf) for a derivation of this update rule). After this update rule, we normalize the scores by dividing by the geometric mean.

Below I show basic python code to do this on the above grid data, initializing $p_A=p_B=p_C=p_D=1$ (same as $b_A=b_B=b_C=b_D=0$ )

In [37]:
import numpy as np

In [38]:
# win data
data=np.array([
    [0,2,5,3],
    [1,0,12,23],
    [17,3,0,4],
    [13,10,7,0]
])

p=np.ones(len(data)) # BT scores, all positive, initialized as ones

def geom_mean(np_arr):
    return np_arr.prod()**(1/len(np_arr))

def bt_iter_idx(idx):
    numerator=(data[idx,:]*p/(p+p[idx])).sum()
    denominator=(data[:,idx]/(p+p[idx])).sum()
    p[idx]=numerator/denominator

def bt_iter():
    global p
    for i in range(len(p)):
        bt_iter_idx(i)
    p/=geom_mean(p)
    
def bt_iter_many(iters):
    for _ in range(iters):
        bt_iter()


bt_iter_many(10)
alpha_string="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
for i,x in enumerate(p):
    print(f"p_{alpha_string[i%26]}: {x:.3f}")
print("------------")
for i,x in enumerate(p):
    print(f"b_{alpha_string[i%26]}: {np.log(x):.3f}")

p_A: 0.346
p_B: 2.597
p_C: 0.873
p_D: 1.277
------------
b_A: -1.063
b_B: 0.954
b_C: -0.136
b_D: 0.244


The above code clearly tells us that the strength order is $B,D,C,A$ which qualitatively fits the data we have. This is the Bradley-Terry Model.

In particular, our original goal was to try and predict $P(A>B)$

We have $p_A = 0.346,p_B=2.597,$ so $P(A>B)=\dfrac{p_A}{p_A+p_B}=\dfrac{0.346}{2.943} \approx 12\%$ under this model


## 4. NLL Loss in Bradley-Terry

Let's return to this equation expressing the BT model $$\begin{align*}P(X>Y)&=\dfrac{e^{b_X}}{e^{b_X}+e^{b_Y}} \\ &= \dfrac{1}{1+e^{-(b_X-b_Y)}} \\ &= \sigma (b_X-b_Y) \end{align*}$$

where $\sigma(x)$ represents the sigmoid function. Now remember, $b_X$ is a score, or reward, associated with entity X.

Therefore, the NLL Loss under the BT model, over some dataset of pairwise outcomes $D$, is $$NLLLoss=-\mathbb{E}_{(X,Y) \sim D}[\log(\sigma(b_X-b_Y))]$$ where X is the preferred outcome, and Y is the non-preferred outcome

This leads to a nice interpretation of pairwise comparisons between responses given by a language model - we can train scores for these responses directly from a dataset of comparisons between them, interpreting the preference probability using the BT model.

## 5. RLHF

Before looking at the details of DPO, let's do a short review of the final stage of RLHF.

In the second stage of of RLHF, we trained a reward model $r_{\phi}$ by minimizing a loss $L(r_{\phi})=-\mathbb{E}_{(x,y_0,y_1) \sim D}[\log(\sigma(r_{\phi}(x,y_1)-r_{\phi}(x,y_0)))]$ where D is some preference dataset. This is exactly the formula we derived in section 4. Essentially, we train the reward model parameters through maximum likelihood estimation of the observed data, interpreted under a B-T model!

In order to stabilize and speed up this process, we borrow the backbone idea commonly seen in image neural networks - see the introduction [here](https://arxiv.org/pdf/2310.19909.pdf). This concept leads us to the idea of taking a backbone from a pretrained language model (namely the embedding and transformer blocks), then attaching a new classifier which can predict a scalar reward from the final hidden state.

This model already deeply understands text due to the backbone, leveraging the power of pretraining, but can be finetuned using the above NLL Loss to predict a scalar reward for text.

After we trained this reward model, we then train our LLM using the reward model to maximize the reward, while respecting a penalty on the KL-divergence between the learned RL policy $\pi_{\theta}$ and the policy $\pi_{SFT}$ obtained from the first step of RLHF, which is supervised finetuning.

The constructed formula for this penalized reward is $$r(x,y)=r_{\phi}(x,y) - \beta * (\log \pi_{\theta}(y|x)-\log \pi_{SFT}(y|x))$$

We then maximize this reward using PPO.

In [the InstructGPT paper](https://arxiv.org/pdf/2203.02155.pdf) an additional factor is added to the PPO gradients, using gradients found from next token prediction over the pretraining dataset. This factor is excluded from the DPO paper and I do not analyze it here.

## 6. Theoretical Analysis of DPO

In RLHF, we use PPO to solve a constrained optimization problem.

The main theoretical breakthrough of the DPO paper shows that an optimal solution to this problem, $\pi^*$, takes a reasonably nice analytical form 

We can then rearrange this equation to get an expression for the reward in terms of the policy itself.

We can then again apply the Bradley-Terry interpretation of response preference probability $P(X>Y)$, but this time using the penalized reward as the score.

When we make this substitution, we get some very nice cancellation, and we get an expression for the likelihood of a preferred response, directly in terms of the policy itself!

This allows us to formulate a new policy objective, that directly trains our policy from the preference dataset itself, without a need to train a reward model.



WIP

## 7. So what happened to the pretraining gradients?

As I mentioned earlier, DPO excludes the pretraining gradients included in InstructGPT which help preserve overall model capability. In this section, we'll examine the evidence whether DPO falls short in this respect.

WIP